Dr Oliviero Andreussi, olivieroandreuss@boisestate.edu

Boise State University, Department of Chemistry and Biochemistry

# Fitting and Data Analysis for the UV-Vis Particle in a Box Experiment {-}

Before we start, let us import the main modules that we will need for this lecture. You may see some new modules in the list below, we will add more details in the right sections.

In [ ]:
# @title Notebook Setup { display-mode: "form" }
# Import the main modules used in this worksheet
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
# Load the google drive with your files
from google.colab import drive
drive.mount('/content/drive')
# The following needs to be the path of the folder with all your datafile in .csv format
base_path = '/content/drive/MyDrive/'

In [ ]:
# @title Functions to load the data { display-mode: "form" }
def load_data_to_file_dict(file_dict):
    """
    Load a uv-vis .csv file.
    The format of the file should have two rows of headers and two columns of data (time and absorbance)
    The file may have additional information from the instrument saved at the bottom

    Input variables:
        file_dict : a dictionary with 'path' and 'name' keys corresponding to the file to be loaded

    Action:
        Add to file_dict a Pandas DataFrame with two columns: time (in seconds) and absorbance (in input units)
    """
    data = pd.read_csv(file_dict['path']+file_dict['name'],header=1,usecols=(0,1)).apply(pd.to_numeric,errors='coerce').dropna()
    data['Abs'] = data['Abs']-data['Abs'].min() # forces the data to be above zero
    if data.keys()[0] == 'Time (sec)' :
        data = data.rename(columns={'Time (sec)':'time'})
        file_dict['xaxis']='time'
    elif data.keys()[0] == 'Time (min)' :
        data['time'] = data['Time (min)'] * 60
        data = data.drop('Time (min)',axis=1)
        file_dict['xaxis']='time'
    elif data.keys()[0] == 'Wavelength (nm)' :
        data = data.rename(columns={'Wavelength (nm)':'wavelength'})
        file_dict['xaxis']='wavelength'
    data = data.rename(columns={'Abs':'absorbance'})
    file_dict['data'] = data
    return

def load_data_to_file_list(file_list):
    """
    Given a list of dictionary files, recursively use load_data_to_file_dict to load the data into each of the dictionaries

    Input variables:
        file_list : a list of dictionary files, each with 'path' and 'name' keys corresponding to the file to be loaded

    Action:
        Add to each file_dict a Pandas DataFrame with two columns: time (in seconds) and temperature (in input units)
    """
    for f in file_list :
        if not ('data' in f): load_data_to_file_dict(f)
    return

def plot_file_dict(file_dict, semilog=False, with_fit=False):
    """
    Given a dictionary file of a bomb calorimetry experiment, plot temperature vs. time.

    Input variables:
        file_dict : a dictionary file with 'path' and 'name' keys corresponding to the file to be loaded
        semilog : use a logarithmic scale for the absorbance axis
        with_fit : plot the fitted curve, if present

    Action:
        Plot absorbance vs. time for the selected file
    """
    fig, ax = plt.subplots()
    if not ('data' in file_dict):
        load_data_to_file_dict(f)
    xaxis = file_dict['xaxis']
    if semilog :
        plt.semilogy(file_dict['data'][xaxis],file_dict['data']['absorbance'],label=file_dict['label'])
        if with_fit and 'absorbance_fitted' in file_dict['data'].keys():
            plt.semilogy(file_dict['data'][xaxis],file_dict['data']['absorbance_fitted'],':',label=file_dict['label']+' fit')
    else:
        plt.plot(file_dict['data'][xaxis],file_dict['data']['absorbance'],label=file_dict['label'])
        if with_fit and 'absorbance_fitted' in file_dict['data'].keys():
            plt.plot(file_dict['data'][xaxis],file_dict['data']['absorbance_fitted'],':',label=file_dict['label']+' fit')
    if xaxis == 'time' :
        plt.xlabel('Time (s)')
    elif xaxis == 'wavelength' :
        plt.xlabel('Wavelength (nm)')
    plt.ylabel('Absorbance (a.u.)')
    plt.legend()
    plt.show()

def plot_file_list(file_list,key='',value=[''],semilog=False,with_fit=False):
    """
    Given a list of dictionary files, plot absorbance vs. time for each file into the same plot.
    If key/value are specified, only plot the files for which the key has the specified value.

    Input variables:
        file_list : a list of dictionary files, each with 'path' and 'name' keys corresponding to the file to be loaded
        key: a string with the name of the key to shortlist the files
        value: the value of the key used to select the shortlist of files

    Action:
        Plot absorbance vs. time for the selected files
    """
    if value == '' or key == '':
        file_shortlist = file_list
    else :
        file_shortlist = [f for f in file_list if f[key] in value ]
    fig, ax = plt.subplots()
    for i,f in enumerate(file_shortlist) :
        color='C'+str(i)
        if not ('data' in f):
            load_data_to_file_dict(f)
        xaxis = f['xaxis']
        if semilog :
            plt.semilogy(f['data'][xaxis],f['data']['absorbance'],color=color,label=f['label'])
            if with_fit and 'absorbance_fitted' in f['data'].keys():
                plt.semilogy(f['data'][xaxis],f['data']['absorbance_fitted'],':',color=color,label=f['label']+' fit')
        else:
            plt.plot(f['data'][xaxis],f['data']['absorbance'],color=color,label=f['label'])
            if with_fit and 'absorbance_fitted' in f['data'].keys():
                plt.plot(f['data'][xaxis],f['data']['absorbance_fitted'],':',color=color,label=f['label']+' fit')
    if xaxis == 'time' :
        plt.xlabel('Time (s)')
    elif xaxis == 'wavelength' :
        plt.xlabel('Wavelength (nm)')
    plt.ylabel('Absorbance (a.u.)')
    plt.legend()
    plt.show()

Set the local path, in case you have absorption spectra that you want to plot

In [ ]:
# @title Set Local Path { display-mode: "form" }
# The following needs to be the path of the folder with all your collected data in .csv format
local_path="Colab Notebooks/ParticleBox_Data/" # @param {type:"string"}
path = base_path+local_path

## Visualize the Systems {-}

The following module needs to be installed on Colab. We won't need it too much for this analysis, but they offer a lot of nice features for chemistry programming. 

In [ ]:
# @title Install and load RDKit { display-mode: "form" }
!pip install rdkit
from rdkit import Chem
from rdkit.Chem import Draw
!pip install cirpy
import cirpy

In particular we can use them to draw the molecules in our experiments. Here are the CAS numbers (note that for one of the molecules CirPy is not able to find the SMILES and you will need to pass it directly):
* 977-96-8
* 605-91-4
* 4727-49-5
* 14187-31-6
* 4727-50-8
* 18300-31-7 ('[I-].CCN1C=C\C(=C/C=C/C=C/c2cc\[n+\](CC)c3ccccc23)c4ccccc14')
* 2197-01-5
* 905-97-5
* 514-73-8
* 3071-70-3
* 905-96-4
* 14806-50-9

In [ ]:
# @title Choose the molecule to draw { display-mode: "form" }
input = '14806-50-9' # @param {type:"string"}
input_type = 'cas' # @param ["smiles", "name", "cas"] {allow-input: true}
if input_type != 'smiles' :
    smiles=cirpy.resolve( input, 'smiles')
else:
    smiles=input
img = Draw.MolToImage( Chem.MolFromSmiles(smiles), size=(300, 300) )
display(img)

Also, if you want to konw the name of one of the molecules, you can use CirPy as follows:

In [ ]:
input = '905-96-4'
cirpy.resolve( input, 'names')

## Plot the Absorption Spectra {-}

If you need to plot the absorption spectra we can use the same functions that were developed for the Kinetics experiment:

In [ ]:
dye0 = {'path':path, 'name':'N3.csv', 'cas' : '977-96-8', 'p' : 3, 'hasO' : 0, 'hasS' : 0, 'label':'N3'}
dye1 = {'path':path, 'name':'C5.csv', 'cas' : '605-91-4', 'p' : 5, 'hasO' : 0, 'hasS' : 0, 'label':'C5'}
dye2 = {'path':path, 'name':'N7.csv', 'cas' : '4727-49-5', 'p' : 7, 'hasO' : 0, 'hasS' : 0, 'label':'N7'}

dyes = [dye0, dye1, dye2]
plot_file_list(dyes)

Note that the plotting command will take care of reading the files and saving the data of each file into the corresponding dictionary, with a key named 'data'. 

In [ ]:
dye0['data'].head()

In order to find the position of the peak, we can use the `argmax()` methods of the `DataFrame` object. This method finds the maximum value in a column and returns its index (NOTE: it does not return the maximum value but its row in the table). Knowing the row, we can look the value of the wavelength at that row to get the quantity we need.

In [ ]:
dye0['lambda_exp'] = dye0['data'].loc[dye0['data']['absorbance'].idxmax(),'wavelength']
print(dye0['lambda_exp'])

We can automate this for all the dyes using a `for` loop:

In [ ]:
for dye in dyes:
    dye['lambda_exp'] = dye['data']['wavelength'].iloc[dye['data']['absorbance'].argmax()]
    print(dye['label'],dye['lambda_exp'])

## Fit the Absorption Wavelengths {-}

In the following we will assume that you collected the values of the absorption wavelength from the instrument, instead of using the Python commands in the previous section. If, instead, you used the notebook to read the spectra and compute `lambda_exp`, you don't need to redefine a new set of dictionaries. 

Let's start by defining a dictionary for some of the molecules in our experiments. Apart from the absorptionn wavelength, these dictionaries will contain some basic information about your molecule (`hasO` and `hasS` refer to the presence of oxygen or sulfur atoms), but you can (should) add any additional characteristic of the molecule that you think may be involved with the absorption wavelength. If you add more features, make sure to add them to each dye and pay attention to avoid typos.

In [ ]:
dye0 = {'cas' : '977-96-8', 'p' : 3, 'hasO' : 0, 'hasS' : 0, 'lambda_exp' : 520}
dye1 = {'cas' : '605-91-4', 'p' : 5, 'hasO' : 0, 'hasS' : 0, 'lambda_exp' : 600}
dye2 = {'cas' : '4727-49-5', 'p' : 7, 'hasO' : 0, 'hasS' : 0, 'lambda_exp' : 590}

As seen in previous notebooks, we can convert a list of dictionaries into a `Pandas.DataFrame`, which will be convenient for math operations and for producing tables.

In [ ]:
dyes = [dye0, dye1, dye2]
dyes_data = pd.DataFrame(dyes)

In [ ]:
print(dyes_data.to_markdown())

## Python Functions {-}

Here we introduce the Python way to define functions. We will use the following function to compute the relation between the absorption wavelength and the conjugation length of the dyes. 

In [ ]:
def lambda_FE(p,alpha=0.):
    """
    Function to compute the absorption wavelenght (in nm)
    as a function of the number of carbon atoms in the conjugated chain
    and an optional parameter alpha that accounts for some wiggle room
    due to the aromatic rings
    """
    return 63.7*(p+3+alpha)**2/(p+4)

Given the function above, we can compute a new column of the dataframe with the predicted lambdas in one command

In [ ]:
dyes_data['lambda_FE']=lambda_FE(dyes_data['p'])
dyes_data

We can visualize the results as usual

In [ ]:
plt.scatter(dyes_data['p'],dyes_data['lambda_exp'])
plt.plot(dyes_data['p'],dyes_data['lambda_FE'],color='red')
plt.xlabel('p')
plt.ylabel('wavelength (nm)')
plt.show()

Given our experimental and predicted values of lambda, we can compute the sum of squares as follows:

In [ ]:
TSS=np.sum((dyes_data['lambda_exp']-dyes_data['lambda_exp'].mean())**2)
print("The total sum of squares is {:10.4f} ".format(TSS))

In [ ]:
RSS=np.sum((dyes_data['lambda_exp']-dyes_data['lambda_FE'])**2)
print("The residual sum of squares is {:10.4f} ".format(RSS))

In [ ]:
R2=1-(RSS/TSS)
print("The coefficient of determination R2 for the particle in a box model is {:5.4f} ".format(R2))

## Finding the Value of Alpha {-}

There are a few different ways we can find the value of $\alpha$ that best reproduces the experimental absorption wavelengths. 

* Given the experimental values of $\lambda$, we could compute the optimal alpha for each individual molecule and than take the average over all the molecules.
* More accurately, we could generate a range of values of $\alpha$ and for each of them compute the $RSS$. The best fit is the one for which the $RSS$ is the lowest. 
* We could linearize the dependence on $p$ in the formula above (i.e., by taking the limit of the formula as $p$ becomes large) and use linear regression to fit the experimental data. From the slope and/or intercept we could estimate $\alpha$. 

Let's try the first approach (but if you want to learn python, try one of the other two as well).

From the formula of $\lambda$ as a function of $\alpha$, we can derive a formula that, given $\lambda$, calculates $\alpha$:

$$\lambda = 63.7\frac{(p+3+\alpha)^2}{(p+4)} \rightarrow \alpha = \sqrt{\frac{\lambda(p+4)}{63.7}}-p-3$$

In [ ]:
dyes_data['alpha']=np.sqrt(dyes_data['lambda_exp']*(dyes_data['p']+4)/63.7)-dyes_data['p']-3

In [ ]:
dyes_data.head()

We can now approximate the optimal value of $\alpha$ to be the average over the molecules we have, and recompute the model's prediction

In [ ]:
alpha_average = dyes_data['alpha'].mean()
dyes_data['lambda_alpha']=lambda_FE(dyes_data['p'],alpha_average)

and visualize the results (NOTE: make sure to add titles and axis labels to your plots)

In [ ]:
plt.scatter(dyes_data['p'],dyes_data['lambda_exp'],label='experiment')
plt.plot(dyes_data['p'],dyes_data['lambda_FE'],color='red',label='particle in a box')
plt.plot(dyes_data['p'],dyes_data['lambda_alpha'],color='green',label='particle in a box + alpha')
plt.xlabel('p')
plt.ylabel('wavelength (nm)')
plt.show()

## Quantum Mechanical Calculations {-}

Most chemistry applications of quantum mechanics (a.k.a. quantum chemistry) relies on a powerful commercial software called Gaussian. This code was first developed by a forefather of quantum chemistry and Nobel prize winner John Pople. However, Gaussian is a Fortran 77 code that requires an expensive license to run. For our applications we can achieve the same results using Python-based codes, at the expense of some computing time. In the following we will be using [PySCF](https://pyscf.org/index.html) for our quantum chemistry calculations, so we will need to install it on our Colab instance.

In [ ]:
!pip install pyscf
!pip install pyscf[geomopt]
from pyscf import gto, scf, lo, tools
from pyscf.geomopt.berny_solver import optimize
#
from scipy.constants import physical_constants # we will need these for units conversion

Let's first go through the main steps of a QC calculation on a simple small molecule, i.e. methane. Before we run any simulation, we need to get some initial guess for the positions of the atoms of our molecule. Luckly we can use `cirpy` to convert our molecule into a `xyz` format, that contains the number of atoms, a comment line, followed by the element and Cartesian coordinates of all the atoms in the molecule. 

In [ ]:
print(cirpy.resolve('methane','xyz'))

For our calculation we only need the atoms information, so we will use some `str`+`list` methods to remove the first two lines from the `xyz` format. 

In [ ]:
xyz = ''.join(string+'\n' for string in cirpy.resolve('methane','xyz').split('\n')[2:])
print(xyz)

We can now create a `Mole` object in PySCF and setup the QC method to use. Part of the accuracy of your calculation will depend on the basis set adopted. The larger the basis set is, the more expensive and (hopefully) more accurate the calculation will be. The available basis sets are listed [here](https://pyscf.org/_modules/pyscf/gto/basis.html). Common choices for small organic molecules go include: `631g`, `631+g*`, `6311g`, and `6311++g**`.  

In [ ]:
mol = gto.M(atom=xyz, basis='631g')

There are a few different methods to choose from: Hartree-Fock ([HF](https://pyscf.org/user/scf.html), cheaper, but not very accurate) and Kohn-Sham Density Functional Theory (DFT, or [KS](https://pyscf.org/user/dft.html)) are the most popular options for organic molecules. There are many different flavors of DFT, each with its own acronym. A list of options is available [here](https://github.com/pyscf/pyscf/blob/master/pyscf/dft/libxc.py), searching for XC_ALIAS. 

In [ ]:
mf = mol.KS()
mf.xc = 'b3lyp' # B3LYP functional, a pretty common choice for organic molecules

The first step in a QC calculation is to optimize (or relax) the positions of the atoms (remember we started from a guess that may be wrong or not in agreement with the QM level of theory we are using). 

In [ ]:
mol_eq = optimize(mf, maxsteps=100)
print(mol_eq.tostring())

Now that we have a more appropriate geometry of our molecule, we can compute the ground state energy and the orbital energies, maybe using a more expensive level of theory.

In [ ]:
mol_eq.basis = '6311++g**'
mf = mol_eq.KS()
mf.xc = 'b3lyp'
mf.kernel()

Once the calculation is finished, the results are all stored inside our object. For example, we can access the orbital energies and occupations.

In [ ]:
print(mf.mo_occ)
print(mf.mo_energy)

Now we can try to apply the same process to our conjugated dyes. NOTE: we need to only select the organic part of the dye, not the iodine atom. To do this, we may want to generate the SMILES of the molecule and manually remove the `I.` part. 

In [ ]:
smile = 'CCN1C(C=Cc2ccccc12)=Cc3ccc4ccccc4[n+]3CC'
xyz = ''.join(string+'\n' for string in cirpy.resolve(smile,'xyz').split('\n')[2:])
print(xyz)

Contrary to the methane example, the dye is positively charged, we need to specify this when we setup the molecule. 

In [ ]:
dye = gto.M(atom=xyz, basis='631g', charge=1, spin=0)

The dyes in our experiments are large molecules and we may not be able to perform QC calculations fast enough for the lab, so we will use cheaper levels of theory (`HF` and `631g`) and no geometry optimization.  

In [ ]:
mf = dye.HF()
mf.kernel()

Once the calculation has finished, we can extract HOMO and LUMO energies and convert the gap into our estimated absorption wavelength.

In [ ]:
lumo = mf.mo_energy[np.where(mf.mo_occ==0)[0][0]] # orbital energies in atomic units (Hartree)
homo = mf.mo_energy[np.where(mf.mo_occ==0)[0][0]-1] # orbital energies in atomic units (Hartree)
Eg = (lumo-homo)*dict(physical_constants)['Hartree energy'][0]
wavelength = dict(physical_constants)['Planck constant'][0] * dict(physical_constants)['speed of light in vacuum'][0] / Eg * 1e9
print(f"The absorption wavelength from the Hartree-Fock/6-31G HOMO-LUMO gap is {wavelength} nm")

## Huckel Model for Conjugated Molecules {-}

While for the Full Report you will only worry about modeling your experiments with a Free Electron (Particle in a Box) kind of model, here we will try to understand how we can use a slightly more advanced quantum mechanical model to predict absorption wavelengths. The Huckel model is a semi-empirical quantum mechanical model that has been developed to describe conjugted systems. While the model relies on some adjustable parameters and involves several possibly strong assumptions, it allows to reproduce some key properties of conjugated and aromatic molecules.

You can learn more on the theory and assumptions of the model in the lectures and in the following online resources: 
* MIT Physical Chemistry on [The Huckel Molecular Orbital Theory](https://dspace.mit.edu/bitstream/handle/1721.1/120336/5-61-fall-2013/contents/lecture-notes/MIT5_61F13_Lecture27-28.pdf)
* P-Chem Lab from Duke University [The Huckel Approximation](https://chem.libretexts.org/Courses/Duke_University/CHEM310L_-_Physical_Chemistry_I_Lab_Manual/04%3A_Absorption_Spectrum_of_Conjugated_Dyes/4.07%3A_Appendix_B_-_The_Huckel_Approximation)
* Columbia Notes on [The Huckel Approximation](http://www.columbia.edu/itc/chemistry/chem-c2407_archive/recitations/huckel.pdf)


The key component of the Huckel model is the fact that the molecular orbitals are univoquely determined by the topology of the conjugated network. The model assumes that we only consider the $p_z$ orbitals of each conjugated atom and we build molecular orbitals from them. As a further approximation, only atomic orbitals that are in connected atoms are allowed to 'interact'. Eventually, we assume that all atoms and bonds are equivalent, so that the relative components of the molecular Hamiltonian are identical. The Hamiltonian of the system can thus be represented as a matrix, where diagonal cells $H_{ii}$ have identical values, while off-diagonal elements $H_{ij}$ are different from zero only if there is a bond between atom $i$ and atom $j$. For carbon based molecules, the model only relies on two parameters $\alpha$ and $\beta$. However, the difference in energy between the electronic states only depends on the latter parameter, which is usually estimated to be $\beta\approx-70.4\; kcal/mol$.

In [ ]:
n_conjugated = 2
alpha = -2
beta = -1
topology = np.zeros((n_conjugated,n_conjugated))
diagonal = np.ones(n_conjugated)*alpha
offdiagonal = np.ones(n_conjugated-1)*beta
topology = topology + np.diag(diagonal,0) + np.diag(offdiagonal,1) + np.diag(offdiagonal,-1)
print(topology)

Molecular orbitals are given by the linear combinations of atomic orbitals that make the Hamiltonian diagonal and minimize the energy of the system. Diagonalization of the Hamiltonian is a linear algebra problem (eigenvalue problem). We can use numpy to compute eigenvalues (a.k.a. orbital energies) and eigenvectors (a.k.a. the values of the coefficients that enter the definition of the molecular orbitals).

In [ ]:
eigenvalues,eigenvectors=np.linalg.eig(topology)
print(eigenvalues)

For the more complicated case of butane we can build the Huckel Hamiltonian as follows:

In [ ]:
n_conjugated = 4
alpha = -2
beta = -1
topology = np.zeros((n_conjugated,n_conjugated))
diagonal = np.ones(n_conjugated)*alpha
offdiagonal = np.ones(n_conjugated-1)*beta
topology = topology + np.diag(diagonal,0) + np.diag(offdiagonal,1) + np.diag(offdiagonal,-1)
print(topology)

which corresponds to the following orbital energies and coefficients

In [ ]:
eigenvalues,eigenvectors=np.linalg.eig(topology)
print(eigenvalues)

You can note that the energies above are all different from the starting energy of the $p_z$ orbitals, which was arbitrary set to $-200\;kcal/mol$. For this kind of Hamiltonians, the eigenvalues can be expressed with an analytical formula $E_j=\alpha+2\beta\cos\left(\frac{\pi}{N+1}J\right)$, with $J=1,2,\dots ,N$. For longer conjugated chains, assuming that we have one electron for each atomic orbital (so number of electrons is $N$ and only the first $N/2$ orbitals are filled), we can estimate the energy difference associated with the absorption maximum as $\Delta E=-4\beta\sin\left(\frac{\pi}{2}\frac{1}{N+1}\right)$. When accounting for the different units, this energy corresponds to a wavelenght of $\lambda^{HMO}=-\frac{28585\;(kcal/mol)}{2\pi\beta\;(kcal/mol)}N$. As for the free electron model, the Huckel Molecular Orbital approach predicts an absorption wavelength that is linear in the number of conjugated centers. The tunable parameter in this case is the $\beta$ constant. 

However, the Huckel model can be automatically extended to account for the aromatic rings and it can, with additional parameters, be extended to include heteroatoms. For example, it is worth looking at the results of the Huckel model for a benzene molecule.

In [ ]:
n_conjugated = 6
alpha = -2.
beta = -1.
topology = np.zeros((n_conjugated,n_conjugated))
diagonal = np.ones(n_conjugated)*alpha
offdiagonal = np.ones(n_conjugated-1)*beta
topology[0,n_conjugated-1]=beta
topology[n_conjugated-1,0]=beta
topology = topology + np.diag(diagonal,0) + np.diag(offdiagonal,1) + np.diag(offdiagonal,-1)
print(topology)

Note that in the following we re-order the results of the eigenvalue problem so that we can see the states in order of their energies (from the largest less stable to the smallest more stable)

In [ ]:
eigenvalues,eigenvectors=np.linalg.eig(topology)
idx = eigenvalues.argsort()[::-1]
eigenvalues = eigenvalues[idx]
eigenvectors = eigenvectors[:,idx]

We can use matplotlib to visualize the coefficients in terms of their signs and magnitudes as follows.

In [ ]:
from matplotlib.colors import BoundaryNorm
bounds = np.arange(np.min(eigenvectors),np.max(eigenvectors),.05)
cmap = plt.get_cmap('seismic')
idx=np.searchsorted(bounds,0)
bounds=np.insert(bounds,idx,0)
norm = BoundaryNorm(bounds, cmap.N)
plt.imshow(eigenvectors,interpolation='none',norm=norm,cmap=cmap)
plt.colorbar()
plt.show()


If you have time, you could try to build a topology matrix for a few of the molecules involved in the experiments and compare the HOMO-LUMO energy difference with the formula reported above for a linear conjugated chain.